### Use LLMs for Reranking

Note: before running below cell, please configure required environment variable `AZURE_OPENAI_API_KEY`, `AZURE_OPENAI_ENDPOINT` by create an `.env` file. Please refer to `../.env.example` as an template.

In [5]:
from openai import AzureOpenAI
from dotenv import load_dotenv
import os
from IPython.display import display, HTML, JSON, Markdown, Image
import textwrap

load_dotenv()
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
OPENAI_GPT4_DEPLOYMENT_NAME = os.getenv("OPENAI_GPT4_DEPLOYMENT_NAME")

client = AzureOpenAI(
  azure_endpoint = AZURE_OPENAI_ENDPOINT, 
  api_key=AZURE_OPENAI_API_KEY,  
  api_version="2024-02-01"
)

In [9]:
def call_openAI(text):
    system_message = """
    The user will enter a question and the possible results from the search query. Please rank these results by relevance to the search query.
    Please answer in JSON format using the schema below.

    schema:
    {
        "documents": {
            "document 1": "Relevance",
            "document 2": "Relevance", 
            ...
        }
    }


    """ 

    response = client.chat.completions.create(
        model=OPENAI_GPT4_DEPLOYMENT_NAME,
        messages = [
            {"role":"system","content":system_message},
            {"role":"user","content":text }
            ],
        temperature=1
    )

    return response.choices[0].message.content

def prettyprint(text: str) -> str:
    print(textwrap.fill(text, 60))

In [12]:
search_results = """
Search Results:
[Document 1] Paul loved going for walks with Mr. McChicken
[Document 2] Paul saw his colleague eat a juicy McDonald’s McChicken burger
[Document 3] Paul loved to eat McDonald’s McChicken burger
[Document 4] Paul always had dinner with Mrs. McChicken
[Document 5] Paul had a lot of lettuce in his salad
"""
question = " Question : What did Paul love to eat? "
answer = call_openAI(question + search_results)
display(HTML(answer))